In [1]:
#必要なパッケージのインポート

import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import reservoirpy as rpy

from scipy.integrate import solve_ivp
import pandas as pd
from reservoirpy.observables import nrmse, rsquare

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import os

rpy.verbosity(0)

from reservoirpy.nodes import Reservoir, Ridge
from reservoirpy.datasets import mackey_glass

# just a little tweak to center the plots, nothing to worry about
from IPython.core.display import HTML
HTML("""
<style>
.img-center {
    display: block;
    margin-left: auto;
    margin-right: auto;
    }
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
    }
</style>
""")

rpy.set_seed(42)

%time


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


CPU times: user 0 ns, sys: 1 µs, total: 1 µs
Wall time: 2.15 µs


In [2]:
def phase_shift(t, shift_hour):
    # 2πは1日に対応します。8時間は(8/24)*2πに対応します。
    shift_value = (shift_hour/24)*2*np.pi
    # 4日ごとに位相をシフトします。
    number_of_shifts = int(t / (4*2*np.pi))
    return number_of_shifts * shift_value


In [3]:
def rossler_system_with_shifted_force(t, state, a, b, c, A, shift_hour):
    x, y, z = state
    dxdt = -y - z + A * np.sin(t + phase_shift(t, shift_hour))  # X項に外力P(t)を加える（位相シフト付き）
    dydt = x + a * y
    dzdt = b + z * (x - c)
    return [dxdt, dydt, dzdt]


In [4]:
# 外力P(t)の振幅Aをパラメータとして追加します。
A = 1.0  # 外力の振幅を設定します。この値を変更することで、外力の強さを変えられます。

# パラメータ
a = 0.2
b = 0.2
c = 5.7

# 初期条件
initial_state = [1.0, 1.0, 1.0]

# 時間の設定
t_span = [0, 10000]  # 開始時刻と終了時刻
t_eval = np.linspace(t_span[0], t_span[1], 100000)  # 評価する時間点

In [5]:
# 位相のシフトする時間を定める．
shift_hours = range(-12, 13)


In [6]:
for shift_hour in shift_hours:
    # 新しいディレクトリを作成
    dir_name = f"data"
    os.makedirs(dir_name, exist_ok=True)
    
    # 微分方程式の数値解を求める
    solution_with_shifted_force = solve_ivp(
        rossler_system_with_shifted_force, t_span, initial_state,
        args=(a, b, c, A, shift_hour), t_eval=t_eval, max_step=0.01
    )

    # 外力P(t)の計算（位相シフト付き）
    P_shifted = A * np.sin(solution_with_shifted_force.t + np.array([phase_shift(ti, shift_hour) for ti in solution_with_shifted_force.t]))

    # DataFrameの作成
    data_frame_with_shifted_force = pd.DataFrame({
        'time': solution_with_shifted_force.t,
        'x': solution_with_shifted_force.y[0],
        'y': solution_with_shifted_force.y[1],
        'z': solution_with_shifted_force.y[2],
        'P_shifted': P_shifted  # 位相シフトされた外力P(t)の列を追加
    })

    # CSVファイル名の設定
    filename_with_force = f"{dir_name}/rossler_data_with_shifted_force_{shift_hour}.csv"

    # CSVファイルにデータを保存
    data_frame_with_shifted_force.to_csv(filename_with_force, index=False)
